In [133]:
#download the datasets from kaggle
!pip install kaggle
!mkdir .kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c su19-final-project
#unzip all the zip files
!unzip \*.zip

mkdir: cannot create directory ‘.kaggle’: File exists


mv: cannot stat 'kaggle.json': No such file or directory
validation_masks.zip: Skipping, found more recently modified local copy (use --force to force download)
validation_images.zip: Skipping, found more recently modified local copy (use --force to force download)
training_validation_Hair_Skin_Masks.zip: Skipping, found more recently modified local copy (use --force to force download)
training_masks.zip: Skipping, found more recently modified local copy (use --force to force download)
training_images.zip: Skipping, found more recently modified local copy (use --force to force download)
testing_images.zip: Skipping, found more recently modified local copy (use --force to force download)
masks2csv.ipynb: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  validation_images.zip
replace validation_images/validation_img_1.jpg? [y]es, [n]o, [A]

In [134]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from skimage import data, io, filters

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

In [0]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
import numpy as np 
import os
import glob
import skimage.io as io
import skimage.transform as trans

def resizeImages(img,mask,flag_multi_class,num_class):
    if(flag_multi_class):
        img = img / 255.0
        mask = mask[:,:,:,0] if(len(mask.shape) == 4) else mask[:,:,0]
        new_mask = np.zeros(mask.shape + (num_class,))
        for i in range(num_class):
            new_mask[mask == i,i] = 1
        new_mask = np.reshape(new_mask,(new_mask.shape[0],new_mask.shape[1]*new_mask.shape[2],new_mask.shape[3])) if flag_multi_class else np.reshape(new_mask,(new_mask.shape[0]*new_mask.shape[1],new_mask.shape[2]))
        mask = new_mask
    elif(np.max(img) > 1):
        img = img / 255.0
        mask = mask /255.0
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

In [0]:
def geneTrainNpy(image_path,mask_path,flag_multi_class = False,num_class = 2,image_prefix = "image",mask_prefix = "mask",image_as_gray = True,mask_as_gray = True):
    image_name_arr = glob.glob(os.path.join(image_path,"%s*.png"%image_prefix))
    image_arr = []
    mask_arr = []
    for index,item in enumerate(image_name_arr):
        img = io.imread(item,as_gray = image_as_gray)
        img = np.reshape(img,img.shape + (1,)) if image_as_gray else img
        mask = io.imread(item.replace(image_path,mask_path).replace(image_prefix,mask_prefix),as_gray = mask_as_gray)
        mask = np.reshape(mask,mask.shape + (1,)) if mask_as_gray else mask
        img,mask = resizeImages(img,mask,flag_multi_class,num_class)
        image_arr.append(img)
        mask_arr.append(mask)
    image_arr = np.array(image_arr)
    mask_arr = np.array(mask_arr)
    return image_arr,mask_arr
def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "grayscale",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = resizeImages(img,mask,flag_multi_class,num_class)
        yield (img,mask)

In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import keras

def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [193]:
from keras.preprocessing.image import ImageDataGenerator
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
model.compile(optimizer = Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
myGene = trainGenerator(2,'./','training_images','training_masks',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
model.fit_generator(myGene,steps_per_epoch=50,epochs=50,callbacks=[model_checkpoint])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/50
Found 1500 images belonging to 1 classes.
Found 1500 images belonging to 1 classes.
50/50 [==============================] - 16s 322ms/step - loss: 0.3544 - acc: 0.8976

Epoch 00001: loss improved from inf to 0.35438, saving model to unet_membrane.hdf5
Epoch 2/50
50/50 [==============================] - 9s 183ms/step - loss: 0.3169 - acc: 0.8900

Epoch 00002: loss improved from 0.35438 to 0.31688, saving model to unet_membrane.hdf5
Epoch 3/50
50/50 [==============================] - 9s 185ms/step - loss: 0.3447 - acc: 0.8826

Epoch 00003: loss did not improve from 0.31688
Epoch 4/50
50/50 [==============================] - 9s 185ms/step - loss: 0.2961 - acc: 0.9095

Epoch 00004: loss improved from 0.31688 to 0.29611, saving model to unet_membrane.hdf5
Epoch 5/50
50/50 [==============================] - 9s 183ms/step - loss: 0.2836 - acc: 0.9095

Epoch 00005: loss improved from 0.29611 to 0.28363, saving model to unet_membrane.hdf5
Epoch 6/50
50/50 [==========================

In [0]:
def testGenerator(test_path,num_image = 927,images_size = (256,256),flag_multi_class = False,as_gray = True):
    for i in range(1, num_image):
        img = io.imread(os.path.join(test_path,"test_img_%d.jpg"%i),as_gray = as_gray)
        img = img / 255.0
        img = trans.resize(img,images_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img
        

In [0]:
def resultsPIC(num_class,color_dict,img):
    img = img[:,:,0] if len(img.shape) == 3 else img
    img_out = np.zeros(img.shape + (3,))
    for i in range(num_class):
        img_out[img == i,:] = color_dict[i]
    return img_out / 255.0
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = resultsPIC(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"test_mask_%d.jpg"%i),img)

In [196]:
results[results>0.5] = 255
test = testGenerator("testing_images")
results = model.predict_generator(test,926,verbose=1)


print(results)

saveResult("drive/My Drive/datatest",results)

926/926 [==============================] - 32s 34ms/step


/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: drive/My Drive/datatest/test_mask_0.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
W0701 12:51:10.145596 140061140498304 util.py:61] Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


[[[[2.71059573e-02]
   [3.04308534e-03]
   [1.02579594e-03]
   ...
   [1.39808059e-02]
   [2.91133523e-02]
   [7.18911886e-02]]

  [[6.12154603e-03]
   [4.04894352e-04]
   [1.30474567e-04]
   ...
   [6.43494725e-03]
   [1.30118132e-02]
   [2.98359692e-02]]

  [[2.79992819e-03]
   [2.88814306e-04]
   [1.64389610e-04]
   ...
   [3.30948830e-03]
   [4.58142161e-03]
   [1.19441450e-02]]

  ...

  [[5.87635636e-02]
   [2.63855457e-02]
   [1.63673759e-02]
   ...
   [1.64226890e-02]
   [1.81061924e-02]
   [3.05961370e-02]]

  [[8.33514333e-02]
   [3.40018570e-02]
   [2.43002772e-02]
   ...
   [1.67115927e-02]
   [2.23968625e-02]
   [4.58756387e-02]]

  [[1.33285940e-01]
   [5.65356314e-02]
   [2.52299309e-02]
   ...
   [3.05060744e-02]
   [5.10273576e-02]
   [1.09300911e-01]]]


 [[[2.85111070e-02]
   [3.25822830e-03]
   [1.12119317e-03]
   ...
   [1.40672326e-02]
   [2.90577114e-02]
   [7.22028017e-02]]

  [[6.58911467e-03]
   [4.51862812e-04]
   [1.47640705e-04]
   ...
   [6.16943836e-03]
 

/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: drive/My Drive/datatest/test_mask_1.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
W0701 12:51:10.157716 140061140498304 util.py:61] Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: drive/My Drive/datatest/test_mask_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
W0701 12:51:10.165281 140061140498304 util.py:61] Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
/usr/local/lib/python3.6/dist-packages/skimage/io/_io.py:141: UserWarning: drive/My Drive/datatest/test_mask_3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
W0701 12:51:10.172924 140061140498304 util.py:61] Lossy conversion from float32 to uint8. Range [0, 1]. Conver

In [197]:
results.shape

(926, 256, 256, 1)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage.transform import resize
from skimage.io import imread
%matplotlib inline  

# encoding function
# based on the implementation: https://www.kaggle.com/rakhlin/fast-run-length-encoding-python/code
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [0]:
# (* update) the input_path using your folder path
input_path = 'drive/My Drive/datatest'

# get a sorted list of all mask filenames in the folder
masks = [f for f in os.listdir(input_path) if f.endswith('.jpg')]
masks = sorted(masks, key=lambda s:int(s.split('_')[2].split('.')[0]))

# encode all masks
encodings = []
for file in masks:
    mask = imread(os.path.join(input_path, file))
    #img_size =10
    mask = resize(mask, (img_size, img_size), mode='constant', preserve_range=True)
    mask = np.array(mask, dtype=np.uint8)
    mask = np.round(mask/255)
    encodings.append(rle_encoding(mask))


# (** update) the path where to save the submission csv file
sub = pd.DataFrame()
sub['ImageId'] = pd.Series(masks).apply(lambda x: os.path.splitext(x)[0])
sub['EncodedPixels'] = pd.Series(encodings).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv(os.path.join('', 'test_seg2.csv'), index=False)